## Transform into podcast starting from a saved json script

In [1]:
import os, json
from pathlib import Path
# import matplotlib.pyplot as plt
os.chdir("../")
Path.cwd()

PosixPath('/home/mymm_psu_gmail_com/FiCast-GPT')

In [2]:
#export environment variables (APIs), as in .example.EnvironmentError
from dotenv import load_dotenv
load_dotenv(override=True)
assert os.getenv("TTS_API_KEY")

In [3]:

from ficast.conversation.podcast import Podcast

Loading config from path: /home/mymm_psu_gmail_com/FiCast-GPT
Loading config version: default from path: ../conf/dialogue


USER_AGENT environment variable not set, consider setting it to identify your requests.


Loading config version: default from path: ../conf/dialogue


In [4]:
with open("notebooks/samples/script.json", 'r') as f:
  chat_result = json.load(f)

In [5]:
my_podcast = Podcast.from_script(chat_result)

Loading config version: default from path: ../conf/dialogue


Loading config version: default from path: ../conf/dialogue


In [6]:
# return chat history from script (warning: this part can fail due to the inconsistency of formatting that may result from individual dialogues from the LLMs)
my_podcast.get_script('raw')
# from ficast.conversation.utils import extract_json_code_block
# extract_json_code_block(my_podcast.raw_script[0].get('content'))

/home/mymm_psu_gmail_com/miniconda3/envs/ficast/lib/python3.11/site-packages/thought_agents/web/summarizer.py:70: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  map_chain = LLMChain(llm=llm, prompt=self.map_prompt)
/home/mymm_psu_gmail_com/miniconda3/envs/ficast/lib/python3.11/site-packages/thought_agents/web/summarizer.py:74: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain/
  combine_chain = StuffDocumentsChain(


[]

In [7]:
my_podcast.get_script('json')

{'topic': 'beer & snacks',
 'abstract': 'Harry Potter and Darth Vader engage in a humorous conversation about snacks, comparing magical treats like Butterbeer and Chocolate Frogs to Sith delights like Death Star-shaped cookies. They bond over nostalgia, with Harry offering a future Butterbeer as a gesture of camaraderie amidst their contrasting worlds of magic and darkness.',
 'participants': {'hosts': [{'name': 'Harry Potter',
    'sex': 'male',
    'description': ''}],
  'guests': [{'name': 'Darth Vader', 'sex': 'male', 'description': ''}]},
 'dialogues': [{'speaker': 'Harry Potter',
   'dialogue': "Hey Vader, have you tried Butterbeer? It's amazing!",
   'inner_thought': 'This is going to be a fun conversation about snacks!'},
  {'speaker': 'Darth Vader',
   'dialogue': 'Butterbeer? Is it strong enough to mask the taste of the Dark Side?',
   'inner_thought': 'I hope he doesn’t think I’m going to enjoy this.'},
  {'speaker': 'Harry Potter',
   'dialogue': "It's not about strength, i

In [8]:
# a json script parsed by `json_parser` agent
# script structure
print(my_podcast.json_script.keys())
# length of conversation
print(len(my_podcast.json_script.get('dialogues')))
# dialogue structure
print(my_podcast.json_script.get('dialogues')[0])

dict_keys(['topic', 'abstract', 'participants', 'dialogues'])
20
{'speaker': 'Harry Potter', 'dialogue': "Hey Vader, have you tried Butterbeer? It's amazing!", 'inner_thought': 'This is going to be a fun conversation about snacks!'}


## Assembling the Podcast Audio Dialogues

In [9]:
%load_ext autoreload
%autoreload 2

from ficast.assembly.ficast import FiCast
from ficast.dialogue.speech import DialogueSynthesis
load_dotenv(override=True)

True

In [10]:
dialoguer_api = DialogueSynthesis(
  client_type="api", 
  base_url=os.getenv("TTS_API_BASE_URL"),
  api_key=os.getenv("TTS_API_KEY")
)
dialoguer_eleven = DialogueSynthesis(
  client_type="elevenlabs", 
  api_key=os.getenv("ELEVENLABS_API_KEY")
)
client = "elevenlabs"
fantasy_ficast = FiCast(
  conversation=my_podcast, dialogue_synthesizer=dialoguer_eleven
)

Token verification successful.
API key provided starting with sk...


In [11]:
my_audio = fantasy_ficast.to_podcast(
  ignore_errors=True, preset="standard")
len(my_audio)

Voice name found. Using Harry Potter voice for Harry Potter
Voice name found. Using Darth Vader voice for Darth Vader


Processing script entries: 100%|██████████| 20/20 [00:34<00:00,  1.74s/it]

Joined 2427 audio segments


1620420

In [19]:
from ficast.dialogue.utils import save_bytes_to_wav, convert_wav_to_mp3
# save audio
from datetime import datetime
# fantasy_ficast.save_podcast(
#   "ficast-outputs/", save_segments=True)
# or

save_bytes_to_wav(my_audio, f"./notebooks/samples/podcasts/{datetime.now()}_{client}.wav")
# with open(fn, 'wb') as file:
#   file.write(my_audio)

ValueError: cannot convert float NaN to integer

In [18]:
convert_wav_to_mp3("notebooks/samples/podcasts/2024-10-05 20:29:00.274315_elevenlabs.wav", "notebooks/samples/podcasts/2024-10-05 20:29:00.274315_elevenlabs.mp3")

ffmpeg version 5.1.6-0+deb12u1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 12 (Debian 12.2.0-14)
  configuration: --prefix=/usr --extra-version=0+deb12u1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librist --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libtwolame --enable-libvidstab --enab

Successfully converted notebooks/samples/podcasts/2024-10-05 20:29:00.274315_elevenlabs.wav to notebooks/samples/podcasts/2024-10-05 20:29:00.274315_elevenlabs.mp3


size=     792kB time=00:01:41.12 bitrate=  64.1kbits/s speed=89.2x    
video:0kB audio:791kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.028010%
